## 问答题
1. 支持向量机的基本思想是什么？
支持向量机(Support Vector Machine，SVM)是一种功能强大且用途广泛的机器学习模型，能够执行线性或非线性分类、回归，甚至异常检测。
2. 什么是支持向量？
SVM训练后，在“街道以外”的地方增加更多的训练实例根本不会对决策边界产生影响，也就是说，它完全由位于街道边缘的实例所决定（或者“支持”）。这些实例被称为支持向量
3. 在使用 SVM 时，缩放输入值为什么很重要？
因为它能确保所有特征在SVM的距离计算和优化过程中具有平等的权重
4. SVM 分类器在对实例进行分类时能输出置信度分数吗？概率呢？
能。不能
5. 你如何在 LinearSVC、SVC 和 SGDClassifier 之间进行选择？
非线性问题选SVC，尤其是样本小的。 线性问题且数据不是特别多时适合LinearSVC。需要弹性网正则化或线性问题中数据太多，需要在线学习时时用SGDClassifier
6. 假设你已经使用 RBF 核训练了一个 SVM 分类器，但它似乎欠拟合训练集。
   你应该增大还是减小 γ（gamma）？C 呢？
增大y ， 增大C
7. ε 不敏感模型是什么意思？
ε 不敏感模型是 SVR 的核心机制，通过引入误差容忍区间 ε，平衡模型的拟合能力与泛化能力
8. 使用核技巧有什么意义？
在不显式将数据映射到高维空间的前提下，实现高维空间中的线性运算

## 编程题
1. 在葡萄酒数据集上训练SVM分类器，可以使用sklearn.datasets.load_wine()加载它。该数据集包含3个不同种植者生产的178个葡萄酒样本的化学分析：目标是训练一个分类模型，该模型能够根据葡萄酒的化学分析预测种植者。由于SVM分类器是二元分类器，将需要使用“一对全部”对所有三个类进行分类。能达到的精度是多少？

   "一对全部"可以复习 **8_sklearn做分类.ipynb**里的笔记，里面提到了用二元分类器做多分类问题

---

2. 提前预习 **10_支持向量机.ipynb** 最新更新的笔记 （把SVM分类用梯度下降实现）； 大概理解笔记后，尝试自己对照笔记 实现用梯度下降实现SVM分类

   并把自定义的SVM分类用于 iris data(鸢尾花数据)； 取花瓣长度 和 花瓣宽度特征， 分类 看是不是 分类2的花 （(iris.target == 2)

   对比下sklearn自带的SVM分类 和 自定义SVM分类 实现的分类效果



In [5]:
from sklearn.datasets import load_wine
import numpy as np
from sklearn.svm import SVC
wine = load_wine(as_frame=True)
X = wine.data
y = wine.target

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
svm_clf = SVC()
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)
accuracy = svm_clf.score(X_test, y_test)
print(accuracy)

0.8055555555555556


In [8]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data[:,2:4]
y = (iris.target==2).astype(int)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
class CustomSVM:
    def __init__(self,learning_rate=0.01,epochs=100,lambda_param=0.01):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.lambda_param = lambda_param
        self.weights = None
        self.bias = None
    def fit(self,X,y):
        n_samples,n_features = X.shape
        y_ = np.where(y==1,0,1)
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.epochs):
            for idx ,x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i,self.weights)+self.bias) >=1
                if condition:
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights)
                else:
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights - np.dot(x_i,y_[idx]))
                    self.bias -= self.lambda_param * y_[idx]
    def predict(self,X):
        approw = np.dot(X,self.weights)+self.bias
        return np.where(approw>0,1,0)


In [11]:
custom_svm = CustomSVM(learning_rate=0.01,epochs=1000, lambda_param=0.01)
custom_svm.fit(X_train,y_train)

In [12]:
y_pred_custom = custom_svm.predict(X_test)

In [13]:
sklearn_svm = SVC(kernel='linear',C=1,random_state=42)
sklearn_svm.fit(X_train,y_train)

,C,1
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [14]:
y_pred_sklearn = sklearn_svm.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score
accuracy_custom = accuracy_score(y_test,y_pred_custom)
accuracy_sklearn = accuracy_score(y_test,y_pred_sklearn)

In [17]:
print(accuracy_custom, accuracy_sklearn)

0.9 1.0
